In [ ]:
import pandas as pd

In [ ]:
# Function to identify the day when max temperature recorded for each city. Return as a dict (city - temp pair)

from pandas import DataFrame

df = pd.read_csv("city_temperature.csv")

def MaxTemp(df:DataFrame):
    dict = {}
    for city in df.columns.drop("Date"):
        filter = df.loc[df[city].idxmax(), ['Date']]
        dict[city] = filter["Date"]
    return dict

print(MaxTemp(df))

    
    


In [ ]:
# Function to identify the monthly average temperature for each of the city. Return dict.

from pandas import DataFrame

df = pd.read_csv("city_temperature.csv")
df['Month'] = pd.to_datetime(df['Date'], dayfirst=True).dt.month_name(locale='English')

def MeanTempMonthWise(df:DataFrame):
    dict = {}
    for city in df.columns.drop("Date").drop("Month"):
        filter = df.groupby("Month")[city].mean().to_dict()
        dict[city] = filter
    return dict

print(MeanTempMonthWise(df))



In [ ]:
# Consider the timeline into 5 days of sliding window. Identify for each city : the number of 5 day stretch(s) 
# where temperature stayed more than that of monthly average. 


from pandas import DataFrame

df = pd.read_csv("city_temperature.csv")
df['Month'] = pd.to_datetime(df['Date'], dayfirst=True).dt.month_name(locale='English')
def count_consecutive_days_above_average(df: DataFrame):
    dict = {}
    for city in df.columns.drop("Date").drop("Month"):
        hotStreaksDict = {}
        avgMeanTempMonthWiseDict = df.groupby("Month")[city].mean().to_dict()    
        for kvp in avgMeanTempMonthWiseDict:
            count = 0
            dx = df[["Month", city]]
            dx = dx[dx["Month"] == kvp]
            chunk_size = 5
            chunks = [dx.iloc[i:i+chunk_size] for i in range(0, len(dx), chunk_size)]
            # print(chunks)
            for ch in chunks:
            # print(ch[city].max())
                if(ch[city].max() > avgMeanTempMonthWiseDict[kvp]):
                    count += 1
            # print(avgMeanTempMonthWiseDict[kvp])
            # print( city, kvp, avgMeanTempMonthWiseDict[kvp], count)
            hotStreaksDict[kvp] = count
        dict[city] = hotStreaksDict
    return dict

print(count_consecutive_days_above_average(df))

    



In [ ]:
# Function that reads the CSV file and returns subject wise topper(s) and overall topper(s), such that:
# Toppers shall have at least 60% attendance
# Toppers shall have the project submitted.


from pandas import DataFrame

df = pd.read_csv("student_scores.csv")

def toppers(df: DataFrame):
    df["Total"] = df["Math"] + df["Science"] + df["English"]
    #print(df)
    subjects = df.columns.drop(['Name','Attendance (%)','Project Submitted', 'Total'])
    #print(subjects)
    subject_wise_topper = {}
    for subject in subjects:
        dx = df.loc[(df["Attendance (%)"] > 60) & (df["Project Submitted"] == True), ["Name", subject]]
        max_mark = dx[subject].max()
        df2 = dx.query(f'{subject} == {max_mark}')
        subject_wise_topper[subject] = df2["Name"].to_list()
    #print(subject_wise_topper)

    dx = df.loc[(df["Attendance (%)"] > 60) & (df["Project Submitted"] == True), ["Name", "Total"]]
    max_mark = dx["Total"].max()
    df2 = dx.query(f'Total == {max_mark}')
    #print(df2.Name.tolist())
    return  (subject_wise_topper, df2.Name.tolist())

(subject_wise_topper, over_all_topper) = toppers(df)
print("Subject wise Toppers:", subject_wise_topper)
print("Overall Toppers:", over_all_topper)

In [ ]:
# A function that returns a data frame with the following columns added along with original data:
# ‘Average Score’ : For each student
# ‘Grade’ :  based on average score (A : >= 90; B : 75 .. 89.99; C : 60 .. 74.99; D : <60)
# ‘Performance’ : 
# ‘Excellent’ : Grade A and attendance > 90%, project submitted
# ‘Needs Attention’ : Grade D OR project not submitted OR attendance < 60%
# ‘Satisfactory’ : All others


from pandas import DataFrame
df = pd.read_csv("student_scores.csv")

def Grade(avgMarks : float):
    if(avgMarks >= 90):
        return "A"
    elif(avgMarks >= 75 and avgMarks <= 89.99):
        return "B"
    elif(avgMarks >=60 and avgMarks <= 74.99):
        return "C"
    else:
        return "D"

def Performance(row):
    if(row["Grade"] == "A" and row["Attendance (%)"] > 90 and row["Project Submitted"] == True):
        return "Excellent"
    elif(row["Grade"] == "D" or row["Project Submitted"] == False or row["Attendance (%)"] < 60):
        return "Attention Needed"
    else:
        return "Satisfactory"

def Avg_Score_Grade_Performance(df: DataFrame):   
    df["Average Score"] = (df["Math"] + df["Science"] + df["English"]) / 3
    df["Grade"] = df["Average Score"].apply(Grade)
    df["Performance"] = df.apply(Performance, axis=1)
    return df

dataframe = Avg_Score_Grade_Performance(df)
print(dataframe)

In [ ]:
# A function that exports the summary statistics of the subject wise marks, attendance to a CSV file.

from pandas import DataFrame
df = pd.read_csv("student_scores.csv")

def Grade(avgMarks : float):
    if(avgMarks >= 90):
        return "A"
    elif(avgMarks >= 75 and avgMarks <= 89.99):
        return "B"
    elif(avgMarks >=60 and avgMarks <= 74.99):
        return "C"
    else:
        return "D"

def Performance(row):
    if(row["Grade"] == "A" and row["Attendance (%)"] > 90 and row["Project Submitted"] == True):
        return "Excellent"
    elif(row["Grade"] == "D" or row["Project Submitted"] == False or row["Attendance (%)"] < 60):
        return "Attention Needed"
    else:
        return "Satisfactory"

def Avg_Score_Grade_Performance(df: DataFrame):   
    df["Average Score"] = (df["Math"] + df["Science"] + df["English"]) / 3
    df["Grade"] = df["Average Score"].apply(Grade)
    df["Performance"] = df.apply(Performance, axis=1)
    return df

dataframe = Avg_Score_Grade_Performance(df)
dataframe.to_csv("student_statistics.csv")